In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
# from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import pandas as pd

books = pd.read_csv("books_cleaned.csv")

In [4]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,2004.0,3.85,247.0,361.0,0,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,2000.0,3.83,241.0,5164.0,0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,1993.0,3.93,512.0,29532.0,0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,2002.0,4.15,170.0,33684.0,0,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,2002.0,4.09,176.0,37569.0,0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,2003.0,2.93,324.0,0.0,0,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,2002.0,3.70,175.0,24.0,0,Journey to the East,9788173031014 This book tells the tale of a ma...
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,2003.0,3.82,198.0,1568.0,0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,1999.0,4.51,531.0,104.0,0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [5]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   sep="\n",
                                   index=False,
                                   header=False,
                                   encoding="utf-8")

In [6]:
raw_documents = TextLoader("tagged_description.txt", encoding="utf-8").load()
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=10_000,
    chunk_overlap=0
)
documents = text_splitter.split_documents(raw_documents)

In [7]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gi

In [8]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="transformers")


In [9]:
# db_books = Chroma.from_documents(
#     documents,
#     embedding=OpenAIEmbeddings()
# )
from langchain_community.embeddings import HuggingFaceEmbeddings

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

db_books = Chroma.from_documents(
    documents,
    embedding=hf_embeddings
)


C:\Users\sksof\AppData\Local\Temp\ipykernel_14900\11285074.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
query = "A book to teach children about nature"
docs = db_books.similarity_search(query, k=10)
docs

[Document(id='c21b68a4-9a30-4f66-b48d-f93e7c280a78', metadata={'source': 'tagged_description.txt'}, page_content='9780060005696 The author of The Battle for Human Nature explains why too much choice has led to the ever increasing complexity of everyday decisions, why too much of a good thing has become detrimental to human psychological and emotional well-being, and how to focus our lives on making the right choices. Reprint. 35,000 first printing.\n9780060009427 What does it mean when a fictional hero takes a journey?. Shares a meal? Gets drenched in a sudden rain shower? Often, there is much more going on in a novel or poem than is readily visible on the surface—a symbol, maybe, that remains elusive, or an unexpected twist on a character—and there\'s that sneaking suspicion that the deeper meaning of a literary text keeps escaping you. In this practical and amusing guide to literature, Thomas C. Foster shows how easy and gratifying it is to unlock those hidden truths, and to discover

In [23]:
books[books["isbn13"] == int(docs[0].page_content.split()[0].strip())]

,isbn13,isbn10,title,authors,categories,thumbnail,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
89,9780060005696,0060005696,The Paradox of Choice,Barry Schwartz,Business & Economics,NaN,2005.0,3.84,265.0,23734.0,0,The Paradox of Choice: Why More Is Less,9780060005696 The author of The Battle for Hum...


In [24]:
def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db_books.similarity_search(query, k=50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]

In [25]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,authors,categories,thumbnail,published_year,average_rating,num_pages,ratings_count,missing_description,title_and_subtitle,tagged_description
89,9780060005696,0060005696,The Paradox of Choice,Barry Schwartz,Business & Economics,NaN,2005.0,3.84,265.0,23734.0,0,The Paradox of Choice: Why More Is Less,9780060005696 The author of The Battle for Hum...
392,9780061208492,0061208493,The Complete C. S. Lewis Signature Classics,C. S. Lewis,Religion,http://books.google.com/books/content?id=JaC0_...,2007.0,4.61,746.0,873.0,0,The Complete C. S. Lewis Signature Classics,9780061208492 Seven Spiritual Masterworks by C...
430,9780064435260,0064435261,A Little Prairie House,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=pRSju...,1999.0,4.19,32.0,1533.0,0,A Little Prairie House,"9780064435260 Long, long ago, a little girl na..."
442,9780067575208,006757520X,The Sense of Wonder,Rachel Carson,Nature,http://books.google.com/books/content?id=Zee5S...,1998.0,4.39,112.0,1160.0,0,The Sense of Wonder,9780067575208 First published more than three ...
1417,9780330490269,0330490265,Dirt Music,Tim Winton,Fiction,http://books.google.com/books/content?id=TAaM1...,2008.0,3.86,465.0,9121.0,0,Dirt Music,"9780330490269 âe~Generous, earthy and raw . . ..."
1744,9780375826894,0375826890,Code Word Kangaroo,George Edward Stanley,Juvenile Fiction,http://books.google.com/books/content?id=k_WF6...,2004.0,4.33,48.0,4.0,0,Code Word Kangaroo,"9780375826894 Gadgets, girls, and bad guys—Ada..."
4100,9780836217698,0836217691,Homicidal Psycho Jungle Cat,Bill Watterson,Humor,http://books.google.com/books/content?id=-aOTQ...,1994.0,4.72,176.0,14952.0,0,Homicidal Psycho Jungle Cat: A Calvin and Hobb...,9780836217698 Reprising the wide-open landscap...
4194,9780881507195,0881507199,King Arthur Flour Whole Grain Baking,NaN,Cooking,http://books.google.com/books/content?id=nXLcP...,2006.0,4.13,544.0,4704.0,0,King Arthur Flour Whole Grain Baking: Deliciou...,9780881507195 A guide to whole grain baking ex...
4441,9781402714597,1402714599,The Secret Garden,Frances Hodgson Burnett,Juvenile Fiction,http://books.google.com/books/content?id=f_9CD...,2004.0,4.13,248.0,553.0,0,The Secret Garden,9781402714597 A ten-year-old orphan comes to l...
4455,9781405201711,1405201711,The Faraway Tree Stories,Enid Blyton,"Children's stories, English",http://books.google.com/books/content?id=2d93O...,1987.0,4.29,583.0,12286.0,0,The Faraway Tree Stories,"9781405201711 Jo, Bessie and Fanny move to the..."
